<a href="https://colab.research.google.com/github/kostiantynkindrat/crime_data_analysis/blob/main/Explanatory_note_(Data_Crime_Analysis).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd

credentials = service_account.Credentials.from_service_account_file('/content/drive/MyDrive/RD/py/Lesson 16/silent-polygon-384520-9abca006c837.json')

project_id = 'silent-polygon-384520'
client = bigquery.Client(credentials=credentials,project=project_id)
# Створюємо датафрейм з вибіркою подій за останні 10 років.
query_text = f"""select *
from `silent-polygon-384520.junior_da_project.crime`
WHERE year >= EXTRACT(YEAR FROM DATE_SUB(CURRENT_DATE(), INTERVAL 10 YEAR))
"""

with client:
    query = client.query(query_text)
    df_crimes = query.to_dataframe()

df_crimes.head()

,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,11013887,JA343598,2017-07-11 12:50:00+00:00,103XX S PRINCETON AVE,0470,PUBLIC PEACE VIOLATION,RECKLESS CONDUCT,OTHER COMMERCIAL TRANSPORTATION,True,False,...,34,49,24,1176122.0,1836217.0,2017,2018-02-10 03:50:01+00:00,41.705935,-87.630664,"(41.7059347, -87.630663516)"
1,12975114,JG143396,2023-02-06 08:34:00+00:00,001XX E 107TH ST,0530,ASSAULT,AGGRAVATED - OTHER DANGEROUS WEAPON,RESIDENCE,True,True,...,9,49,04A,1179244.0,1834054.0,2023,2023-02-13 03:50:44+00:00,41.699929,-87.619297,"(41.699928657, -87.61929671)"
2,11145889,JA507262,2017-11-10 10:45:00+00:00,103XX S INDIANA AVE,0650,BURGLARY,HOME INVASION,RESIDENCE,False,False,...,9,49,05,1179414.0,1836257.0,2017,2018-02-10 03:50:01+00:00,41.705970,-87.618607,"(41.705970121, -87.618607299)"
3,11172776,JA542510,2017-12-07 09:00:00+00:00,009XX E 104TH ST,1200,DECEPTIVE PRACTICE,STOLEN PROP: BUY/RECEIVE/POS.,APARTMENT,False,False,...,9,50,13,1184238.0,1836191.0,2017,2018-02-10 03:50:01+00:00,41.705678,-87.600944,"(41.705677782, -87.600944364)"
4,10899417,JA210912,2017-04-03 01:35:00+00:00,104XX S MICHIGAN AVE,1506,PROSTITUTION,SOLICIT ON PUBLIC WAY,STREET,True,False,...,9,49,16,1178870.0,1835516.0,2017,2018-02-10 03:50:01+00:00,41.703949,-87.620622,"(41.703949088, -87.620621835)"


In [ ]:
# Аналізуємо датасет на наявність null значень
for column in df_crimes.columns:
    null_count = df_crimes[column].isnull().sum()
    print(f'{column}: {null_count} null values')

unique_key: 0 null values
case_number: 0 null values
date: 0 null values
block: 0 null values
iucr: 0 null values
primary_type: 0 null values
description: 0 null values
location_description: 9661 null values
arrest: 0 null values
domestic: 0 null values
beat: 0 null values
district: 1 null values
ward: 59 null values
community_area: 5 null values
fbi_code: 0 null values
x_coordinate: 39698 null values
y_coordinate: 39698 null values
year: 0 null values
updated_on: 0 null values
latitude: 39680 null values
longitude: 39680 null values
location: 39680 null values


In [ ]:
# Оскільки наявність null значень у всіх колонках свідчить про неповноту або не коректність даних, ми можемо їх видалити.
df_crimes = df_crimes.dropna()

In [1]:
# Перевіримо датасет на наявність null значень
for column in df_crimes.columns:
    null_count = df_crimes[column].isnull().sum()
    print(f'{column}: {null_count} null values')

NameError: name 'df_crimes' is not defined

In [ ]:
# Підрахуємо к-ть унікальних та загальну кількість значень у кожному стовпці. Це нам дасть розуіміння про можливі викиди.
for column in df_crimes.columns:
    unique_count = df_crimes[column].nunique()
    total_count = df_crimes[column].count()
    difference = total_count - unique_count

pivot1 = []
for column in df_crimes.columns:
    unique_count = df_crimes[column].nunique()
    total_count = df_crimes[column].count()
    difference = total_count - unique_count
    pivot1.append({'column': column, 'unique Count': unique_count, 'total Count': total_count, 'difference': difference})

pivot1_df = pd.DataFrame(pivot1)
pivot1_df = pivot1_df.set_index('column')
print(pivot1_df)

                      unique Count  total Count  difference
column                                                     
unique_key                 2652520      2652520           0
case_number                2652220      2652520         300
date                        963082      2652520     1689438
block                        36512      2652520     2616008
iucr                           374      2652520     2652146
primary_type                    35      2652520     2652485
description                    510      2652520     2652010
location_description           200      2652520     2652320
arrest                           2      2652520     2652518
domestic                         2      2652520     2652518
beat                           275      2652520     2652245
district                        23      2652520     2652497
ward                            50      2652520     2652470
community_area                  77      2652520     2652443
fbi_code                        26      

Аналізуємо ресурс [Police Areas & Beats](https://chicagopolicedept-my.sharepoint.com/personal/gisteam_chicagopolice_org/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fgisteam%5Fchicagopolice%5Forg%2FDocuments%2FCPD%20PDFs%2Farea%5Fdistrict%5Fbeat%5F11x17%2Epdf&parent=%2Fpersonal%2Fgisteam%5Fchicagopolice%5Forg%2FDocuments%2FCPD%20PDFs&ga=1) для того, щоб перевірити коректність значень у колонках "beat" та  "district".

In [ ]:
#Створюємо функцію за допомогою якої ми можемо вивести усі унікальні значення необхідної колнки.
def display_unique_values(df_crimes, column):
    unique_values = df_crimes[column].unique()
    print(f'Unique values in column {column}: {unique_values}')

In [ ]:
display_unique_values(df_crimes, 'district')

Unique values in column district: <IntegerArray>
[ 5, 10, 18,  8,  3, 22, 16, 11,  6, 24,  1, 19, 14,  9,  4, 17, 12,  7, 25,
  2, 20, 15, 31]
Length: 23, dtype: Int64


Якщо вірити ресурсу [Police Areas & Beats](https://chicagopolicedept-my.sharepoint.com/personal/gisteam_chicagopolice_org/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fgisteam%5Fchicagopolice%5Forg%2FDocuments%2FCPD%20PDFs%2Farea%5Fdistrict%5Fbeat%5F11x17%2Epdf&parent=%2Fpersonal%2Fgisteam%5Fchicagopolice%5Forg%2FDocuments%2FCPD%20PDFs&ga=1),

<br> 1) district-а (району) "31" не існує.
<br> 2) Перші 2 цифри колонки "beat" повинні бути значенням колонки "district".
<br> Вважаю, що колонка "beat" має більші шанси на коректність, зважаючи на 1-й пункт. Відштовхуюсь від неї. Перевіримо чи є не співпадіння у даних, зважаючи на пункт 2.

In [ ]:
#Механічний спосіб перевірки значень.
#df_crimes_district = df_crimes[df_crimes['district'] == 1]
#df_crimes_district[['district','beat']]

#Створюємо маску для автоматичної перевірки.
mask = ((df_crimes['beat'].astype(str).str.len() == 3) & (df_crimes['district'] == df_crimes['beat'].astype(str).str[0].astype(int))) | \
       ((df_crimes['beat'].astype(str).str.len() == 4) & (df_crimes['district'] == df_crimes['beat'].astype(str).str[:2].astype(int)))

# Обчислюємо к-ть значень False
fals_district_count = len(df_crimes) - mask.sum()

print(f'Кількість значень False: {fals_district_count}')

Кількість значень False: 336


In [ ]:
# Створюємо функцію для перевірки та редагування значень у стовпці district
def check_district(row):
    beat = str(row['beat'])
    district = row['district']

    if len(beat) == 3:
        calculated_district = int(beat[0])
    elif len(beat) == 4:
        calculated_district = int(beat[:2])
    else:
        calculated_district = None

    if calculated_district is not None and district != calculated_district:
        row['district'] = calculated_district

    return row

df_crimes = df_crimes.apply(check_district, axis=1)

In [ ]:
#Перевіимо ще раз.
mask1 = ((df_crimes['beat'].astype(str).str.len() == 3) & (df_crimes['district'] == df_crimes['beat'].astype(str).str[0].astype(int))) | \
       ((df_crimes['beat'].astype(str).str.len() == 4) & (df_crimes['district'] == df_crimes['beat'].astype(str).str[:2].astype(int)))

fals_district_count = len(df_crimes) - mask1.sum()

print(f'Кількість значень False: {fals_district_count}')

Кількість значень False: 0


In [ ]:
display_unique_values(df_crimes, 'district')

Unique values in column district: [ 5 10 18  8  3 16 11  6 24  1 19 14  9  4 22 17 12  7 25  2 20 15]


In [ ]:
target_table = "silent-polygon-384520.junior_da_project.My_crime"
job_location = "US"
df_crimes.to_gbq(target_table, project_id=project_id, if_exists='replace',
            location=job_location, progress_bar=True, credentials=credentials)
num_rows=len(df_crimes)
print(f'''
Exported {num_rows} rows to BigQuery table
'''
)

100%|██████████| 1/1 [00:00<00:00, 1136.98it/s]


Exported 2652520 rows to BigQuery table



Тепер перевіримо таку умову: якщо значення колонки "location" однакове, то значення у колонках "beat", "district", "ward", "community_area" теж мають бути однакові.

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd

credentials = service_account.Credentials.from_service_account_file('/content/drive/MyDrive/RD/py/Lesson 16/silent-polygon-384520-9abca006c837.json')

project_id = 'silent-polygon-384520'
client = bigquery.Client(credentials=credentials,project=project_id)

query_text = f"""select *
from `silent-polygon-384520.junior_da_project.My_crime`
"""

with client:
    query = client.query(query_text)
    df_crimes = query.to_dataframe()
df_crimes.head()

,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,latitude,longitude,location,time_of_day,weekday,month,date_month_start,unemployment_rate,house_price_index,president
0,12038952,JD218880,2020-04-28 07:35:00+00:00,031XX W 47TH ST,1090,ARSON,ATTEMPT ARSON,STREET,True,False,...,41.808138,-87.703072,"41.808138193, -87.703072372",night,Tuesday,4,2020-04-01 00:00:00+00:00,18.4,136.111433,Donald Trump
1,25013,JD203044,2020-04-06 09:54:00+00:00,021XX S CENTRAL PARK AVE,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,True,False,...,41.852326,-87.715019,"41.852325522, -87.715019384",night,Monday,4,2020-04-01 00:00:00+00:00,18.4,136.111433,Donald Trump
2,12294219,JE143282,2021-02-16 10:30:00+00:00,053XX N KIMBALL AVE,1710,OFFENSE INVOLVING CHILDREN,ENDANGERING LIFE / HEALTH OF CHILD,RESIDENCE,False,True,...,41.978210,-87.713605,"41.978209511, -87.71360452",night,Tuesday,2,2021-02-01 00:00:00+00:00,7.5,145.954473,Joe Biden
3,26111,JE146705,2021-02-20 09:04:00+00:00,072XX S JEFFERY BLVD,0110,HOMICIDE,FIRST DEGREE MURDER,ALLEY,True,True,...,41.764106,-87.576257,"41.764106016, -87.576256836",night,Saturday,2,2021-02-01 00:00:00+00:00,7.5,145.954473,Joe Biden
4,12302563,JE153733,2021-02-27 06:37:00+00:00,024XX S WHIPPLE ST,1020,ARSON,BY FIRE,VEHICLE NON-COMMERCIAL,False,False,...,41.847737,-87.701458,"41.847737402, -87.701457979",night,Saturday,2,2021-02-01 00:00:00+00:00,7.5,145.954473,Joe Biden


In [ ]:
#Групуємо дані за значеннями у стовпці "location"
grouped_location = df_crimes.groupby(['location'])

# Створюємо маску для перевірки умови
mask2 = grouped_location['beat'].transform('nunique') == 1
mask2 &= grouped_location['district'].transform('nunique') == 1
mask2 &= grouped_location['ward'].transform('nunique') == 1
mask2 &= grouped_location['community_area'].transform('nunique') == 1

# Обчислюємо к-ть значень False
false_location_count = (~mask2).sum()

print(f'Кількість рядків, які не відповідають умовам: {false_location_count}')

Кількість рядків, які не відповідають умовам: 865884


На мою думку, занадто велике число рядків, щоб це могло бути суцільним викидом. Спробуємо звірити по черзі, можливо дійдем до якогось логічного висновку.

In [ ]:
# Спочатку звіряємо колонки 'location' та 'beat'
mask_beat  = grouped_location['beat'].transform('nunique') == 1

# Обчислюємо к-ть рядків, які не відповідають умові
false_location_beat_count = (~mask_beat ).sum()

print(f'Кількість рядків, які не відповідають умовам: {false_location_beat_count}')

Кількість рядків, які не відповідають умовам: 246202


In [ ]:
false_location_beat_rows = df_crimes.loc[~mask_beat, ['location', 'beat']]
print(false_location_beat_rows)

                              location  beat
7        (41.851034751, -87.705231765)  1024
15       (41.844241776, -87.725461425)  1031
19       (41.839809174, -87.717106897)  1032
23       (41.845323703, -87.695231543)  1033
30       (41.692358646, -87.623096746)   522
...                                ...   ...
2698360  (41.997717137, -87.690492381)  2412
2698423  (41.902582485, -87.731840051)  1111
2698427  (41.875159738, -87.710819098)  1133
2698430  (41.998072486, -87.670939306)  2432
2698474  (41.893623014, -87.765347251)  1524

[246202 rows x 2 columns]


Рандомно вибираємо декілька значень колонки 'location' та перевіряємо як саме не сходяться значення у колонці 'beat'

In [ ]:
mask = df_crimes['location'] == '(41.851034751, -87.705231765)'
rows = df_crimes.loc[mask, ['location', 'beat']]
print(rows)

                              location  beat
7        (41.851034751, -87.705231765)  1024
25539    (41.851034751, -87.705231765)  1033
224998   (41.851034751, -87.705231765)  1024
313884   (41.851034751, -87.705231765)  1033
532953   (41.851034751, -87.705231765)  1024
713537   (41.851034751, -87.705231765)  1024
837154   (41.851034751, -87.705231765)  1024
936013   (41.851034751, -87.705231765)  1024
1169948  (41.851034751, -87.705231765)  1024
1274407  (41.851034751, -87.705231765)  1024
2118616  (41.851034751, -87.705231765)  1024
2261621  (41.851034751, -87.705231765)  1024


In [ ]:
mask = df_crimes['location'] == '(41.997717137, -87.690492381)'
rows = df_crimes.loc[mask, ['location', 'beat']]
print(rows)

                              location  beat
758441   (41.997717137, -87.690492381)  2412
851469   (41.997717137, -87.690492381)  2412
1013710  (41.997717137, -87.690492381)  2412
1034361  (41.997717137, -87.690492381)  2412
1046341  (41.997717137, -87.690492381)  2412
1069283  (41.997717137, -87.690492381)  2412
1260943  (41.997717137, -87.690492381)  2412
1299991  (41.997717137, -87.690492381)  2412
1379465  (41.997717137, -87.690492381)  2412
1394379  (41.997717137, -87.690492381)  2412
1718179  (41.997717137, -87.690492381)  2412
1838150  (41.997717137, -87.690492381)  2413
1880038  (41.997717137, -87.690492381)  2412
1982155  (41.997717137, -87.690492381)  2412
2082748  (41.997717137, -87.690492381)  2412
2259673  (41.997717137, -87.690492381)  2412
2635871  (41.997717137, -87.690492381)  2412
2698360  (41.997717137, -87.690492381)  2412


In [ ]:
mask = df_crimes['location'] == '(41.893623014, -87.765347251)'
rows = df_crimes.loc[mask, ['location', 'beat']]
print(rows)

                              location  beat
34748    (41.893623014, -87.765347251)  1524
104202   (41.893623014, -87.765347251)  1524
145300   (41.893623014, -87.765347251)  1524
172860   (41.893623014, -87.765347251)  1524
259791   (41.893623014, -87.765347251)  1524
...                                ...   ...
2596758  (41.893623014, -87.765347251)  1524
2633549  (41.893623014, -87.765347251)  1524
2676310  (41.893623014, -87.765347251)  1524
2684937  (41.893623014, -87.765347251)  1511
2698474  (41.893623014, -87.765347251)  1524

[85 rows x 2 columns]


In [ ]:
mask = df_crimes['location'] == '(41.692358646, -87.623096746)'
rows = df_crimes.loc[mask, ['location', 'beat']]
print(rows)

                              location  beat
30       (41.692358646, -87.623096746)   522
4110     (41.692358646, -87.623096746)   522
7189     (41.692358646, -87.623096746)   522
9239     (41.692358646, -87.623096746)   522
13276    (41.692358646, -87.623096746)   522
...                                ...   ...
2577722  (41.692358646, -87.623096746)   522
2582979  (41.692358646, -87.623096746)   522
2608863  (41.692358646, -87.623096746)   522
2620006  (41.692358646, -87.623096746)   531
2682381  (41.692358646, -87.623096746)   522

[390 rows x 2 columns]


Проаналізувавши значення колонки 'beat', можемо припустити, що значення колонки 'location' (координати) знаходяться на межі двох "бітів". Щоб у цьому пересвідчитись скористаємось ресурсом [Police Areas & Beats](https://chicagopolicedept-my.sharepoint.com/personal/gisteam_chicagopolice_org/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fgisteam%5Fchicagopolice%5Forg%2FDocuments%2FCPD%20PDFs%2Farea%5Fdistrict%5Fbeat%5F11x17%2Epdf&parent=%2Fpersonal%2Fgisteam%5Fchicagopolice%5Forg%2FDocuments%2FCPD%20PDFs&ga=1).
<br> Переглянувши вищезазначений ресурс, бачимо, що наша гіпотеза була вірною.
<br>
<br> Таким ж чином перевіримо колонку 'district'.

In [ ]:
pd.set_option('display.max_rows', 25)

In [ ]:
mask_district = grouped_location['district'].transform('nunique') == 1

false_location_district_count = (~mask_district).sum()

print(f'Кількість рядків, які не відповідають умовам: {false_location_district_count}')

Кількість рядків, які не відповідають умовам: 48155


In [ ]:
false_location_district_rows = df_crimes.loc[~mask_district, ['location', 'beat', 'district']]
print(false_location_district_rows)

                              location  beat  district
35       (41.852118283, -87.679579111)  1034        10
116        (41.7960115, -87.703627606)   822         8
154      (41.776303146, -87.607096088)   321         3
171      (41.758528974, -87.608498691)   323         3
198      (41.759335603, -87.559654632)   334         3
...                                ...   ...       ...
2698186  (41.937961113, -87.717350016)  2523        25
2698251  (41.758169024, -87.628719577)   731         7
2698271  (41.888267493, -87.627075047)   111         1
2698331  (41.735994703, -87.649098615)   613         6
2698423  (41.902582485, -87.731840051)  1111        11

[48155 rows x 3 columns]


In [ ]:
mask = df_crimes['location'] == '(41.937961113, -87.717350016)'
rows = df_crimes.loc[mask, ['location', 'beat','district']]
print(rows)

                              location  beat  district
979051   (41.937961113, -87.717350016)  2523        25
1595847  (41.937961113, -87.717350016)  1412        14
2228909  (41.937961113, -87.717350016)  2523        25
2657449  (41.937961113, -87.717350016)  2523        25
2694857  (41.937961113, -87.717350016)  2523        25
2695148  (41.937961113, -87.717350016)  2523        25
2698186  (41.937961113, -87.717350016)  2523        25


In [ ]:
mask = df_crimes['location'] == '(41.852118283, -87.679579111)'
rows = df_crimes.loc[mask, ['location', 'beat','district']]
print(rows)

                              location  beat  district
35       (41.852118283, -87.679579111)  1034        10
27625    (41.852118283, -87.679579111)  1034        10
341076   (41.852118283, -87.679579111)  1034        10
389027   (41.852118283, -87.679579111)  1034        10
496569   (41.852118283, -87.679579111)  1034        10
513040   (41.852118283, -87.679579111)  1034        10
914736   (41.852118283, -87.679579111)  1234        12
967059   (41.852118283, -87.679579111)  1034        10
997599   (41.852118283, -87.679579111)  1034        10
1051432  (41.852118283, -87.679579111)  1034        10
1067482  (41.852118283, -87.679579111)  1034        10
1079904  (41.852118283, -87.679579111)  1034        10
1084683  (41.852118283, -87.679579111)  1034        10
1660972  (41.852118283, -87.679579111)  1234        12
1667383  (41.852118283, -87.679579111)  1034        10
1876064  (41.852118283, -87.679579111)  1034        10


In [ ]:
mask = df_crimes['location'] == '(41.902582485, -87.731840051)'
rows = df_crimes.loc[mask, ['location', 'beat','district']]
print(rows)

                              location  beat  district
67697    (41.902582485, -87.731840051)  1111        11
125834   (41.902582485, -87.731840051)  2534        25
141024   (41.902582485, -87.731840051)  2534        25
156995   (41.902582485, -87.731840051)  1111        11
190484   (41.902582485, -87.731840051)  2534        25
...                                ...   ...       ...
2530426  (41.902582485, -87.731840051)  1111        11
2602036  (41.902582485, -87.731840051)  1111        11
2624035  (41.902582485, -87.731840051)  2534        25
2630431  (41.902582485, -87.731840051)  1111        11
2698423  (41.902582485, -87.731840051)  1111        11

[77 rows x 3 columns]


Знову звертаємось до [Police Areas & Beats](https://chicagopolicedept-my.sharepoint.com/personal/gisteam_chicagopolice_org/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fgisteam%5Fchicagopolice%5Forg%2FDocuments%2FCPD%20PDFs%2Farea%5Fdistrict%5Fbeat%5F11x17%2Epdf&parent=%2Fpersonal%2Fgisteam%5Fchicagopolice%5Forg%2FDocuments%2FCPD%20PDFs&ga=1) і знову бачимо, що значення колонки 'location' (координати) знаходяться на межі вже двох "районів".
<br>
<br> Отже робимо висновок, що дані коректні.
<br>
<br> Наступний крок - добавляємо нові колонки у датасет, які допоможуть нам у визначенні цікавих метрик, таких як:
<br>- Пора доби, в яку був зареєстрований злочин.
<br>- Місяць та день тиждень за яких були вчинені злочини.
<br>- Рівень резробіття у Чікаго за цей період часу.
<br>- Рівень цін на житло у Чікаго на цей період часу.
<br>- Роки президентства полтіиків, за каденції яких були вчинені злочини.

In [ ]:
# Визначаємо тип даних у колонці "date"
df_crimes['date'].dtype

datetime64[ns, UTC]

In [ ]:
# Створюємо функцію, яка згрупує дані на 4 типи пори доби.
def get_time_of_day(hour):
    if 6 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 18:
        return 'afternoon'
    elif 18 <= hour < 24:
        return 'evening'
    else:
        return 'night'

# Переводимо колонку "date" у часовий пояс Чікаго, для того, щоб ми мали розуміння про те, який час був саме у Чікаго на момент скоєння злочину.
df_crimes['date'] = df_crimes['date'].dt.tz_convert('America/Chicago')

# Створюємо нову колонку 'time_of_day'
df_crimes['time_of_day'] = df_crimes['date'].dt.hour.apply(get_time_of_day)
df_crimes.head()

,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,longitude,location,time_of_day,weekday,month,date_month_start,unemployment_rate,house_price_index,president,date_24h
0,12374387,JE240306,2021-02-27 06:00:00-06:00,012XX S MICHIGAN AVE,5000,OTHER OFFENSE,OTHER CRIME AGAINST PERSON,APARTMENT,False,False,...,-87.624134,"(41.866578077, -87.624134464)",morning,Saturday,2,2021-02-01 00:00:00+00:00,7.5,145.954473,Joe Biden,2021-02-27 06:00:00
1,12027025,JD205514,2020-04-09 05:30:00-05:00,067XX S JEFFERY BLVD,1581,SEX OFFENSE,NON-CONSENSUAL DISSEMINATION OF PRIVATE SEXUAL...,RESIDENCE,False,False,...,-87.576474,"(41.772719038, -87.576474498)",night,Thursday,4,2020-04-01 00:00:00+00:00,18.4,136.111433,Donald Trump,2020-04-09 05:30:00
2,12037018,JD216432,2020-04-20 07:00:00-05:00,069XX S GREEN ST,1710,OFFENSE INVOLVING CHILDREN,ENDANGERING LIFE / HEALTH OF CHILD,RESIDENCE,False,True,...,-87.645711,"(41.768104798, -87.645711322)",morning,Monday,4,2020-04-01 00:00:00+00:00,18.4,136.111433,Donald Trump,2020-04-20 07:00:00
3,12024782,JD203036,2020-04-06 03:12:00-05:00,071XX S STATE ST,0330,ROBBERY,AGGRAVATED,CONVENIENCE STORE,False,False,...,-87.624890,"(41.764797653, -87.624889913)",night,Monday,4,2020-04-01 00:00:00+00:00,18.4,136.111433,Donald Trump,2020-04-06 03:12:00
4,12288244,JE135306,2021-02-06 22:17:00-06:00,069XX S WABASH AVE,0497,BATTERY,AGGRAVATED DOMESTIC BATTERY - OTHER DANGEROUS ...,APARTMENT,False,True,...,-87.623370,"(41.768303717, -87.623370395)",evening,Saturday,2,2021-02-01 00:00:00+00:00,7.5,145.954473,Joe Biden,2021-02-06 22:17:00


In [ ]:
# Під час візуалізації даних я замітив, що рядків зі значенням "afternoon" немає у датасеті, що досить дивно. Це означає, що жоден злочин не був вчинений в період 12:00-18:00 (за часом у Чікаго). Перевіримо це
df_crimes['date'] = pd.to_datetime(df_crimes['date'])
mask_afternoon = (df_crimes['date'].dt.hour >= 12) & (df_crimes['date'].dt.hour <= 18)
afternoon_df = df_crimes[mask_afternoon]
print(afternoon_df)

Empty DataFrame
Columns: [unique_key, case_number, date, block, iucr, primary_type, description, location_description, arrest, domestic, beat, district, ward, community_area, fbi_code, x_coordinate, y_coordinate, year, updated_on, latitude, longitude, location, time_of_day, weekday, month, date_month_start, unemployment_rate, house_price_index, president, date_24h]
Index: []

[0 rows x 30 columns]


Дійсно, бачимо, що жоден злочин не був зареєстрований у цей часовий проміжок. <br> P.S. Також я перевірив це у GBQ за допомогою SQL, там також не було знайдено жодного такого рядка.

In [ ]:
# Перевіримо датасет на наявність пустих значень або пробілів
for column in df_crimes.columns:
  if df_crimes[column].dtype == 'object':
    blank_count = (df_crimes[column].str.strip() == '').sum()
    print(f"Кількість порожніх рядків або пробілів у стовпці {column}: {blank_count}")

Кількість порожніх рядків або пробілів у стовпці case_number: 0
Кількість порожніх рядків або пробілів у стовпці block: 0
Кількість порожніх рядків або пробілів у стовпці iucr: 0
Кількість порожніх рядків або пробілів у стовпці primary_type: 0
Кількість порожніх рядків або пробілів у стовпці description: 0
Кількість порожніх рядків або пробілів у стовпці location_description: 0
Кількість порожніх рядків або пробілів у стовпці fbi_code: 0
Кількість порожніх рядків або пробілів у стовпці location: 0
Кількість порожніх рядків або пробілів у стовпці time_of_day: 0


In [ ]:
# Створюємо нову колонку 'weekday' з днем тижня
df_crimes['weekday'] = df_crimes['date'].dt.day_name()

# Створюємо нову колонку 'month' з місяцем скоєння злочину
df_crimes['month'] = df_crimes['date'].dt.month

# Створюємо нову колонку 'year' з роком скоєння злочину
df_crimes['year'] = df_crimes['date'].dt.year
df_crimes.head()

,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location,time_of_day,weekday,month
0,12923823,JF511748,2022-12-14 20:30:00-06:00,066XX W 59TH ST,0650,BURGLARY,HOME INVASION,RESIDENCE,False,False,...,1133289.0,1864691.0,2022,2023-01-03 03:46:28+00:00,41.784925,-87.786856,"(41.784925254, -87.786855618)",evening,Wednesday,12
1,9649982,HX300283,2014-06-05 05:00:00-05:00,023XX W 95TH ST,1122,DECEPTIVE PRACTICE,COUNTERFEIT CHECK,OTHER,False,False,...,1162539.0,1841588.0,2014,2018-02-10 03:50:01+00:00,41.720967,-87.680254,"(41.720967, -87.680254496)",night,Thursday,6
2,12241234,JD454405,2020-12-05 23:00:00-06:00,060XX N KEELER AVE,1110,DECEPTIVE PRACTICE,BOGUS CHECK,RESIDENCE - PORCH / HALLWAY,False,False,...,1147256.0,1939916.0,2020,2020-12-17 03:44:58+00:00,41.991096,-87.733718,"(41.991096068, -87.733718465)",evening,Saturday,12
3,9547880,HX200132,2014-03-26 05:50:00-05:00,058XX N SPAULDING AVE,1365,CRIMINAL TRESPASS,TO RESIDENCE,RESIDENCE PORCH/HALLWAY,False,True,...,1153318.0,1938648.0,2014,2016-02-04 06:33:39+00:00,41.987498,-87.711455,"(41.987497976, -87.711454613)",night,Wednesday,3
4,9059337,HW204266,2013-03-23 02:45:00-05:00,007XX E 111TH ST,1350,CRIMINAL TRESPASS,TO STATE SUP LAND,POLICE FACILITY/VEH PARKING LOT,True,False,...,1183356.0,1831503.0,2013,2018-02-10 03:50:01+00:00,41.692834,-87.604319,"(41.692833841, -87.60431945)",night,Saturday,3


Першим очевидним показником дял аналізу рівня злочинності є рівень безробіття ["Unemployment Rate" in Chicago-Naperville-Elgin, IL-IN-WI (MSA)](https://fred.stlouisfed.org/series/CHIC917URN).
<br> Дані пораховані на період 1-го числа кожного місяця. Отже нам потрібно створити нову колонку "date_month_start" для того щоб мати змогу поєднати дві таблиці.

In [ ]:
chicago_unemployment_rate = pd.read_csv('/content/drive/MyDrive/RD/Course Project/Chicago_unempl.csv')
#print(chicago_unemployment_rate)

chicago_unemployment_rate.columns = chicago_unemployment_rate.columns.str.lower()
chicago_unemployment_rate['date'] = pd.to_datetime(chicago_unemployment_rate['date'])
mask = (chicago_unemployment_rate['date'].dt.year >= 2013) & (chicago_unemployment_rate['date'].dt.year <= 2023)
chicago_unemployment_rate=chicago_unemployment_rate.loc[mask]
#print(chicago_unemployment_rate)

chicago_unemployment_rate.columns = ['date_month_start', 'unemployment_rate']

chicago_unemployment_rate.head()

,date_month_start,unemployment_rate
276,2013-01-01,10.2
277,2013-02-01,10.0
278,2013-03-01,9.3
279,2013-04-01,8.9
280,2013-05-01,9.0


Також доволі цікавим показником, з яким цікаво подивитись залежність, є - [Home Price Index (High Tier) for Chicago, Illinois](https://fred.stlouisfed.org/series/CHXRHTSA), який показу рівень цін на житло у Чікаго за певнйи період часу.

In [ ]:
chicago_home_price = pd.read_csv('/content/drive/MyDrive/RD/Course Project/home_price_index.csv')
#print(chicago_house_price)

chicago_home_price.columns = chicago_home_price.columns.str.lower()
chicago_home_price['date'] = pd.to_datetime(chicago_home_price['date'])
mask = (chicago_home_price['date'].dt.year >= 2013) & (chicago_home_price['date'].dt.year <= 2023)
chicago_home_price=chicago_home_price.loc[mask]
#print(chicago_unemployment_rate)

chicago_home_price.columns = ['date_month_start', 'house_price_index']

chicago_home_price.head()

,date_month_start,house_price_index
252,2013-01-01,117.349445
253,2013-02-01,117.641993
254,2013-03-01,118.317255
255,2013-04-01,119.181888
256,2013-05-01,121.100505


In [ ]:
df_crimes['date_month_start'] = df_crimes['date'].dt.to_period('M').dt.to_timestamp().dt.date
#df_crimes.head()
df_crimes['date_month_start'] = pd.to_datetime(df_crimes['date_month_start'])

merged_df_crimes = pd.merge(df_crimes, chicago_unemployment_rate, how='left', on='date_month_start')
merged_df_crimes_2 = pd.merge(merged_df_crimes, chicago_home_price, how='left', on='date_month_start')

merged_df_crimes_2.head()

<ipython-input-42-22832d3044f9>:1: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df_crimes['date_month_start'] = df_crimes['date'].dt.to_period('M').dt.to_timestamp().dt.date


,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,updated_on,latitude,longitude,location,time_of_day,weekday,month,date_month_start,unemployment_rate,house_price_index
0,10188503,HY376479,2015-07-28 12:00:00+00:00,020XX W EVERGREEN AVE,1752,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,RESIDENCE,False,True,...,2018-02-10 03:50:01+00:00,41.905992,-87.678449,"(41.905992382, -87.678449338)",morning,Tuesday,7,2015-07-01,6.1,128.464749
1,12753561,JF289173,2022-06-21 08:15:00+00:00,016XX N ROCKWELL ST,1780,OFFENSE INVOLVING CHILDREN,OTHER OFFENSE,APARTMENT,False,True,...,2023-01-03 03:46:28+00:00,41.910984,-87.692120,"(41.910984456, -87.692120187)",night,Tuesday,6,2022-06-01,5.0,170.169985
2,10680307,HZ428941,2016-09-10 11:44:00+00:00,024XX W ALTGELD ST,1020,ARSON,BY FIRE,STREET,False,False,...,2018-02-10 03:50:01+00:00,41.926714,-87.688060,"(41.926713582, -87.688060102)",morning,Saturday,9,2016-09-01,5.5,130.452710
3,24562,JC281250,2019-05-28 12:25:00+00:00,018XX N MONTICELLO AVE,0110,HOMICIDE,FIRST DEGREE MURDER,PARK PROPERTY,True,False,...,2022-09-18 04:45:51+00:00,41.914092,-87.717899,"(41.914092032, -87.717899225)",morning,Tuesday,5,2019-05-01,3.5,135.410891
4,9864591,HX514438,2014-11-20 10:50:00+00:00,015XX N MILWAUKEE AVE,2250,LIQUOR LAW VIOLATION,LIQUOR LICENSE VIOLATION,RESTAURANT,True,False,...,2016-02-04 06:33:39+00:00,41.909536,-87.676154,"(41.909535792, -87.676153748)",night,Thursday,11,2014-11-01,6.1,128.364827


In [ ]:
merged_df_crimes_2['date'] = pd.to_datetime(merged_df_crimes_2['date']).dt.tz_localize(None)
merged_df_crimes_2['date'] = pd.to_datetime(merged_df_crimes_2['date'])

def get_president(date):
    if date >= pd.to_datetime('2021-01-20'):
        return 'Joe Biden'
    elif date >= pd.to_datetime('2017-01-20'):
        return 'Donald Trump'
    elif date >= pd.to_datetime('2009-01-20'):
        return 'Barack Obama'
    else:
        return 'Other'

merged_df_crimes_2['president'] = merged_df_crimes_2['date'].apply(get_president)
merged_df_crimes_2.head()

,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,latitude,longitude,location,time_of_day,weekday,month,date_month_start,unemployment_rate,house_price_index,president
0,10188503,HY376479,2015-07-28 12:00:00,020XX W EVERGREEN AVE,1752,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,RESIDENCE,False,True,...,41.905992,-87.678449,"(41.905992382, -87.678449338)",morning,Tuesday,7,2015-07-01,6.1,128.464749,Barack Obama
1,12753561,JF289173,2022-06-21 08:15:00,016XX N ROCKWELL ST,1780,OFFENSE INVOLVING CHILDREN,OTHER OFFENSE,APARTMENT,False,True,...,41.910984,-87.692120,"(41.910984456, -87.692120187)",night,Tuesday,6,2022-06-01,5.0,170.169985,Joe Biden
2,10680307,HZ428941,2016-09-10 11:44:00,024XX W ALTGELD ST,1020,ARSON,BY FIRE,STREET,False,False,...,41.926714,-87.688060,"(41.926713582, -87.688060102)",morning,Saturday,9,2016-09-01,5.5,130.452710,Barack Obama
3,24562,JC281250,2019-05-28 12:25:00,018XX N MONTICELLO AVE,0110,HOMICIDE,FIRST DEGREE MURDER,PARK PROPERTY,True,False,...,41.914092,-87.717899,"(41.914092032, -87.717899225)",morning,Tuesday,5,2019-05-01,3.5,135.410891,Donald Trump
4,9864591,HX514438,2014-11-20 10:50:00,015XX N MILWAUKEE AVE,2250,LIQUOR LAW VIOLATION,LIQUOR LICENSE VIOLATION,RESTAURANT,True,False,...,41.909536,-87.676154,"(41.909535792, -87.676153748)",night,Thursday,11,2014-11-01,6.1,128.364827,Barack Obama


In [ ]:
# Відформатуємо колонку "location", прибравши дужки на початку та у кінці.
df_crimes['location'] = df_crimes['location'].apply(lambda x: x.strip('()'))
df_crimes.head()

,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,latitude,longitude,location,time_of_day,weekday,month,date_month_start,unemployment_rate,house_price_index,president
0,12298548,JE148882,2021-02-22 02:00:00+00:00,048XX S DORCHESTER AVE,1155,DECEPTIVE PRACTICE,AGGRAVATED FINANCIAL IDENTITY THEFT,RESIDENCE,False,False,...,41.807236,-87.591936,"41.807235907, -87.591935773",evening,Sunday,2,2021-02-01 00:00:00+00:00,7.5,145.954473,Joe Biden
1,12287498,JE134396,2021-02-06 02:55:00+00:00,069XX S CHAPPEL AVE,0326,ROBBERY,AGGRAVATED VEHICULAR HIJACKING,STREET,False,False,...,41.769008,-87.575154,"41.769008286, -87.575154463",evening,Friday,2,2021-02-01 00:00:00+00:00,7.5,145.954473,Joe Biden
2,12032860,JD212160,2020-04-19 03:34:00+00:00,088XX S BLACKSTONE AVE,141B,WEAPONS VIOLATION,UNLAWFUL USE - OTHER FIREARM,RESIDENCE - YARD (FRONT / BACK),False,False,...,41.734361,-87.588074,"41.73436095, -87.588073849",evening,Saturday,4,2020-04-01 00:00:00+00:00,18.4,136.111433,Donald Trump
3,12297992,JE148202,2021-02-10 12:01:00+00:00,079XX S MARYLAND AVE,1750,OFFENSE INVOLVING CHILDREN,CHILD ABUSE,APARTMENT,False,False,...,41.750377,-87.603944,"41.750377017, -87.60394359",morning,Wednesday,2,2021-02-01 00:00:00+00:00,7.5,145.954473,Joe Biden
4,12030733,JD209765,2020-04-15 06:35:00+00:00,018XX N ROCKWELL ST,0920,MOTOR VEHICLE THEFT,ATTEMPT - AUTOMOBILE,STREET,False,False,...,41.914596,-87.692216,"41.914596401, -87.692216301",night,Wednesday,4,2020-04-01 00:00:00+00:00,18.4,136.111433,Donald Trump


Цікаво дізнатись у який день було вчинено найбільше злочинів? Давайте подивимось

In [ ]:
daily_crime_count = df_crimes.groupby(df_crimes['date'].dt.date).size().sort_values(ascending=False)
daily_crime_count.head()

date
2020-05-31    1893
2013-01-01    1276
2013-05-01    1152
2013-09-01    1092
2013-10-01    1076
dtype: int64

31 травня 2020 року. Причиною такої кількості вчинених злочинів, йомовірно, є протести по всій Америці після подій, пов'язаних з Джорджем Флойдом, який був убитий поліцейським за декілька днів до цієї дати.

#<br> Висновки.
<br> Даний курсовий проект був розроблений на підставі завдання, виданого керівниками навчальної програми - Chicago Crime Data.
<br> За посиланням у [Looker Studio](https://lookerstudio.google.com/reporting/22f91b79-0d20-4ce9-932f-37e5e31a79ab) представлена візуалізація до даного датасету.
<br>
<br> У розділі "Group & filters" можна знати інструментарій для різноманітних потреб та запитів, щоб ви мали змогу знайти відповідь на питання яке вас цікавить.
<br> - Arrested - фільтр, що дзволяє відібрати лише дані у яких відбувся арешт злочинця.
<br> - Domestic - фільтр, що дозволяє відібрати побутові злочини.
<br> - President - фільтр, що дозволяє переглянути дані за період каденції вибраного президента Сполучених Штатів Америки (чисто фанова опція насправді, добавив заради особистої цікавості :) )
<br> - Year - дозволяє вибрати дані, відфільтровані за роками.
<br> - Ward - список усіх виборчих округів міста Чікаго
<br> - Community area - список усіх районів міста Чікаго.
<br> - District - список усіх поліцейських округів міста Чікаго.
<br> - Beat - список поліцейських кварталів міста Чікаго.
<br> - FBI code - Crime Types with Definition in Federal Bureau of Investigation's.
<br> - IUCR - llinois Uniform Crime Reporting Codes.
<br> - Primary type - Primary description in IUCR.
<br> - Secondary type - Secondary description in IUCR.
<br>
<br> Crimes Dynamic - Графік, який демонструє динаміку скоєння злочинів за вибраний проміжок часу. Також на графіку представлені конверсію на те, чи відбувся арешт та на наявність побутовісті злочинів. З цих даних можна побачити, що конвесії в арешт та побутовість злочину, практично, дзеркальні, тобто, вірогідність того, що відбудеться арешт, якщо злочин є побутовим, набагато нижча.

<br>
<br> Top 10 regions and types of crimes - графік, на якому представлені регіони (beat/ district/ community area/ ward (за вибором)) у рядках та частку конкретного виду злочину (IUCR/Primary type/Secondary type) у накописувальних барах.
<br>
<br> Count of crimes to unemployment rate corelation - Кореляція кількості злочинів до рівня безробіття у Чікаго. Можемо побачити, що загальна тенденція зберігається на обох графіках, окрім подій початку Covid 19, коли безробіття вийшло на найвищий рівень, а рівень злочинності отримав нанижчий показник.
<br>
<br> Count of crimes to house price index corelation - Кореляція кількості злочинів до індексу цін на житло у Чікаго. За цим графіком можна побачити, що до 2020 року при поступовому підвищенні індекса цін на житло - відбувався поступовий спад рівня злочинності, а вже після 2020-го року тенденція змінилась і зараз ці графіки йдуть в одному напрямку.
<br>
<br> Top 20 crime locations - графік, схожий до "Top 10 regions and types of crimes", але відрізняється даними у рядках, тут зображені місця скоєння злочину.
<br>
<br> Далі зображені 2 кругових діаграми та 1 стовпчикову, які поділяють час скоєння злочину на дні тижня (Day of week breakdown), пори доби (Time of day breakdown) та місяць (Top crime months). Тут ми можемо звернути увагу на такі дані:
<br> - Найбільш злочинний день тижня - п'ятниця (моя здогадка у тому, що  люди люблять розслаблятись після важкого робочго тижня і рівень вживання алкоголю та наркотичних речовин є більшим ніж у інші дні, що приводить до різних небажаних наслідків), а найменш злочинний - неділя (я думаю, через те, що багато людей є віруючими і у більшості релігіях, неділя - день молитви, люди ходять у церкву, слухають мотиваційні проповіді і це впливає на кількість скоєння злочинів :) ).
<br> Також можна зауважити, що день поділений лише на 3 пори, хоча, розподіляючи дані, я вказував 4 (ще afternoon - злочин, зареєстрований з 12:00 по 18:00), що може вказати на відсутність злочинів у afternoon або на наявний викид у базі даних.
<br> На стовпчиковій діаграмі ми можемо звернути увагу, що теплі весняні та літні місяці є більш злочинними ніж холодні зимові та осінні.

<br> Chicago crime map дозволяє нам побачити локацію, на якій відбувався той чи інший цікавий нам злочин.
<br>
<br> Далі зображені 3 графіки політичнох тематики (чисто моя цікавість), а саме:
<br> - Crimes dynamic by president of USA - Динаміка злочинів по каденція кожного з президентів, що були на посаді під час періоду, який ми аналізуємо. Можемо побачити, що за каденції Дональда Трампа рівень злочинності потихеньку йшов на спад, а після приходу пана Джо Байдена - навпаки, почав рости.
<br> Також ми маємо 2 графіки "Correlation of the number of crimes to the percentage of votes cast for candidate Trump by ward (term of Trump/ Biden)" на яких ми можемо прослідкувати залежність кількості злочинів у одному окрузі (ward) та часткою голосів в підтримку Дональда Трампа до виборів президента у цьому ж окрузі. Перший графі показує рівень злочинності після 2016 року, як Трамп став президентом, а другий після 2020 року, коли Джо байден став президентом. На обох графіках присутня цікава тенденція - що чим більше підтримки Дональда Трампа, тим менший рівень злочинності у цьому окрузі.
<br> Підводячи підсумки по цим 3-м графікам, можна сказати, що "демократи" не впливають позитивно на зменшення кількості злочинів.
<br>
